NER (с учётом выбранной категории - музыкальных

Для выделения названий продукта можно использовать:

1) Составить предположительный список инструментов и других товаров (guitar, strings, pick, ukulele, etc.), потом использовать модель w2v обученную на отзывах чтобы получить больше вариантов упоминаний (upd после проверки, в целом кажется, что нет большой разницы между обучением на отзывах и предобученной модели из списка)
- плюсы: ничего не надо кроме самого корпуса
- минусы: надо подобрать список, не удалось быстро придумать способ ввести дву- трёх- словные наименования (возможно он есть простой и интуитивный, и если так очень сильно бы улучшил полезность выдачи, потому что пока она выдаёт скорее более полные наименования чем важные характеристики)

2) Использовать stanza, deeppavlov или другой инструмент, в котором уже есть правила извлечения наименования продукта (можно отсортировать по частотности, чтобы некоторые ошибочные продукты пропали) - этот способ круче всего, но оказался ужасно долгим, ускорить не смогла, отказалась

3) Выделить ключевые слова в отзывах (только существительные) и использовать их

4) Написать паттерн упоминаний типа "I like this X", "This is a great X", "I (don't) recommend this X" etc. (мб подойдёт даже просто "This X")

5) Скачать мета дату и брать название продукта оттуда, а потом использовать его как целиком так и по частям (кажется для музыкальных инструментов плохо подойдёт, на первый взгляд чаще называют тип инструмента чем модель)

In [1]:
import pandas as pd
import gzip
import json
import re

def parse(path):
    g = gzip.open(path)
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [2]:
df = getDF('Musical_Instruments_5.json.gz')

In [3]:
reviews = []
for line in range(len(df['reviewText'])):
     reviews += [df['reviewText'][line]]
reviews = [str(r) for r in reviews]
reviews_n = []
for rev in reviews:
    reviews_n += [re.sub(r'[^\w\s]', '', rev.lower()).split()]

In [4]:
mini_dict = ['piano', 'guitar', 'ukulele', 'drum', 'sax', 'bass', 'instrument', 'string']

In [5]:
import gensim.downloader
from gensim.models import Word2Vec

#print(list(gensim.downloader.info()['models'].keys()))

rev_vectors = Word2Vec(sentences=reviews_n)

In [6]:
words = list(rev_vectors.wv.vocab)

['its', 'good', 'for', 'beginners', 'i', 'recommend', 'this', 'starter', 'ukulele', 'kit', 'has', 'everything', 'you', 'need', 'to', 'learn', 'the', 'gdaughter', 'received', 'christmas', 'present', 'last', 'year', 'and', 'plays', 'if', 'often', 'according', 'my', 'order', 'history', 'bought', 'towards', 'end', 'of', '2012', 'as', 'a', 'black', 'friday', 'week', 'lightning', 'deal', 'im', 'writing', 'nearly', 'four', 'years', 'later', 'with', 'any', 'stringed', 'instrument', 'strings', 'really', 'needed', 'stretch', 'out', 'before', 'they', 'would', 'stay', 'in', 'tune', 'so', 'first', 'several', 'days', 'it', 'was', 'play', 'few', 'minutes', 'then', 'tuning', 'daily', 'after', 'weeks', 'uke', 'held', 'great', 'tuner', 'is', 'not', 'included', 'package', 'which', 'bulk', 'minus', 'star', 'because', 'real', 'beginner', 'wouldnt', 'have', 'way', 'thing', 'either', 'or', 'reference', 'note', 'say', 'on', 'piano', 'keyboard', 'can', 'download', 'decent', 'app', 'though', 'buck', 'do', 'your

In [7]:
instrs = set()
for w in mini_dict:
    instrs.add(w)
    for instr in rev_vectors.most_similar(w):
        instrs.add(instr[0])

<ipython-input-7-4490a33cb3e8>:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  for instr in rev_vectors.most_similar(w):


In [8]:
instrs

{'5string',
 'acoustic',
 'alto',
 'axe',
 'banjo',
 'bari',
 'bass',
 'basses',
 'batter',
 'bow',
 'casio',
 'cello',
 'clarinet',
 'classical',
 'dobro',
 'drum',
 'drums',
 'e',
 'electric',
 'feeler',
 'flute',
 'fret',
 'fretless',
 'g',
 'grand',
 'gretsch',
 'gstring',
 'guitar',
 'guitarbass',
 'guitars',
 'harmonica',
 'hihat',
 'horn',
 'instrument',
 'instruments',
 'kala',
 'keybed',
 'keyboard',
 'kick',
 'ludwig',
 'luna',
 'mandolin',
 'mouthpiece',
 'nut',
 'p',
 'piano',
 'pianos',
 'resonator',
 'sampled',
 'sax',
 'saxophone',
 'snare',
 'soprano',
 'steinway',
 'sting',
 'stings',
 'string',
 'strings',
 'tama',
 'tenor',
 'tom',
 'toms',
 'trombone',
 'trumpet',
 'uke',
 'ukelele',
 'ukulele',
 'violin',
 'wound',
 'yamaha'}

In [9]:
colls = []
for instr in instrs:
    for rev in reviews_n:
        rev = ' '.join(rev)
        if instr in rev:
            entr = re.findall("\s\w*?\s"+instr, rev)
            entr += re.findall(instr+"\s\w*?\s", rev)
            for i in range(len(entr)):
                colls.append(entr[i].strip().split())

In [10]:
colls[:20]

[['my', 'dobro'],
 ['my', 'dobro'],
 ['dobro', 'acoustic'],
 ['the', 'dobro'],
 ['dobro', 'have'],
 ['or', 'dobro'],
 ['dobro', 'its'],
 ['guitar', 'dobro'],
 ['dobro', 'mandolin'],
 ['slim', 'dobro'],
 ['dobro', 'i'],
 ['body', 'dobro'],
 ['hifi', 'dobro'],
 ['pickup', 'dobro'],
 ['perfect', 'dobro'],
 ['dobro', 'beautiful'],
 ['dobro', 'sound'],
 ['dobro', 'you'],
 ['and', 'dobro'],
 ['dobro', 'it']]

In [11]:
import nltk
from nltk.collocations import *

In [12]:
# Коллекция метрик
bigram_measures = nltk.collocations.BigramAssocMeasures()
# Экстрактор коллокаций
finder2 = BigramCollocationFinder.from_documents(colls)

# Применить фильтр по частоте
finder2.apply_freq_filter(10)
ignored_words = nltk.corpus.stopwords.words('english')
finder2.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)

In [13]:
# Про корреляцию
from nltk.metrics.spearman import *

In [14]:
scores_ll = finder2.score_ngrams(bigram_measures.likelihood_ratio)
scores_pmi = finder2.score_ngrams(bigram_measures.pmi)
scores_dice = finder2.score_ngrams(bigram_measures.dice)
scores_stud = finder2.score_ngrams(bigram_measures.student_t)

In [15]:
list(ranks_from_scores(scores_ll))[:20]

[(('acoustic', 'guitar'), 0),
 (('electric', 'guitar'), 1),
 (('tom', 'power'), 2),
 (('tom', 'line'), 3),
 (('guitar', 'guitar'), 4),
 (('classical', 'guitar'), 5),
 (('electric', 'guitars'), 6),
 (('acoustic', 'guitars'), 7),
 (('bass', 'drum'), 8),
 (('alto', 'sax'), 9),
 (('snare', 'drum'), 10),
 (('daddario', 'string'), 11),
 (('guitar', 'player'), 12),
 (('strings', 'guitar'), 13),
 (('drum', 'machine'), 14),
 (('guitar', 'center'), 15),
 (('fret', 'board'), 16),
 (('daddario', 'strings'), 17),
 (('musical', 'instrument'), 18),
 (('grand', 'piano'), 19)]

In [16]:
list(ranks_from_scores(scores_pmi))[:10]

[(('gretsch', 'catalina'), 0),
 (('ludwig', 'breakbeats'), 1),
 (('gretsch', 'electromatic'), 2),
 (('ludwig', 'keystone'), 3),
 (('feeler', 'gauges'), 4),
 (('kala', 'ka15s'), 5),
 (('kala', 'mkt'), 6),
 (('alto', 'zmx862'), 7),
 (('casio', 'px150'), 8),
 (('casio', 'privia'), 9)]

In [17]:
list(ranks_from_scores(scores_dice))[:10]

[(('tom', 'power'), 0),
 (('alto', 'sax'), 1),
 (('tom', 'line'), 2),
 (('feeler', 'gauges'), 3),
 (('tenor', 'sax'), 4),
 (('grand', 'piano'), 5),
 (('flat', 'wound'), 6),
 (('casio', 'privia'), 7),
 (('ludwig', 'keystone'), 8),
 (('alto', 'saxophone'), 9)]

In [18]:
list(ranks_from_scores(scores_stud))[:10]

[(('acoustic', 'guitar'), 0),
 (('electric', 'guitar'), 1),
 (('acoustic', 'guitars'), 2),
 (('classical', 'guitar'), 3),
 (('electric', 'guitars'), 4),
 (('tom', 'power'), 5),
 (('bass', 'drum'), 6),
 (('guitar', 'strap'), 7),
 (('guitar', 'player'), 8),
 (('wound', 'strings'), 9)]

In [19]:
def res_printer(word, scores):
    print(word + '\n' + '-----')
    for line in scores:
        if word in line[0][0] or word in line[0][1]:
            print(line[0][0] + ' ' + line[0][1])

In [20]:
res_printer('guitar', scores_stud)

guitar
-----
acoustic guitar
electric guitar
acoustic guitars
classical guitar
electric guitars
guitar strap
guitar player
guitar center
bass guitar
playing guitar
play guitar
guitar stand
guitar case
guitar picks
guitar players
new guitar
guitar strings
every guitar
classical guitars
guitar amp
local guitar
travel guitar
guitar straps
guitar cable
expensive guitar
different guitars
different guitar
favorite guitar
taylor guitar
two guitars
cheap guitar
many guitar
style guitar
guitar shop
beginner guitar
many guitars
played guitar
guitar stands
several guitars
guitar sounds
size guitar
bass guitars
guitar pick
another guitar
lead guitar
expensive guitars
fender guitar
guitar without
guitar body
guitar rig
starter guitar
acousticelectric guitar
guitar came
regular guitar
guitar tech
guitar effects
guitar amps
guitar store
sounding guitar
several guitar
resonator guitar
parlor guitar
guitar cases
martin guitar
multiple guitars
guitar string
beautiful guitar
fender guitars
guitar teacher

In [21]:
res_printer('string', scores_stud)

string
-----
wound strings
daddario string
daddario strings
new string
coated string
steel string
new strings
nylon string
coated strings
best string
gauge string
wound string
strings ive
change string
great string
elixir string
changing string
change strings
changing strings
elixir strings
stringed instrument
favorite string
gauge strings
best strings
sounding string
favorite strings
guitar strings
good string
sounding strings
great strings
light string
steel strings
nylon strings
quality string
aquila string
strings last
aquila strings
bronze string
martin string
light strings
bronze strings
ball string
stringed instruments
string winder
martin strings
ball strings
strings sound
six string
different string
string height
good strings
string set
stock string
quality strings
string broke
uncoated string
stock strings
heavier string
string changes
uncoated strings
extra string
goto string
string sets
string tension
strings make
nice string
better string
goto strings
open string
strings s

In [22]:
res_printer('ukulele', scores_stud)

ukulele
-----
soprano ukulele
tenor ukulele
baritone ukulele
concert ukulele
ukulele strings
first ukulele
banjo ukulele
ukulele player
kala ukulele
size ukulele
cheap ukulele
starter ukulele
tenor ukuleles
new ukulele
electric ukulele
inexpensive ukulele
ukulele tuner
toy ukulele
little ukulele
soprano ukuleles
ukulele string
play ukulele
ukulele perfectly
expensive ukulele
ukulele gig
ukulele tuning
beginner ukulele
ukulele players
ukulele case
ukulele group
ukulele banjo
makala ukulele
ukulele bag
acousticelectric ukulele
luna ukulele
soprano ukulelea
wood ukulele
ukulele fits
ukulele club
beautiful ukulele
standard ukulele
level ukulele
violin ukulele
ukulele also
sounding ukulele
ukulele stand
two ukulele
quality ukulele
ukulele strap
playing ukulele
ukulele would
nice ukulele
ukulele sounds
best ukulele
ukulele ive
ukulele sound
good ukulele
great ukulele
bass ukulele
guitars ukulele
ukulele electric
ukulele bass
acoustic ukulele
string ukulele
guitar ukulele
ukulele guitar


In [23]:
res_printer('flute', scores_stud)

flute
-----
american flute
spirit flute
flute player
flute peg
song flute
play flute
flute bass
guitar flute


In [24]:
res_printer('drum', scores_stud)

drum
-----
bass drum
snare drum
kick drum
drum set
drum machine
drum kit
drum head
drum heads
electronic drum
snare drums
drum key
drum sticks
drum machines
drum kits
real drum
drum tracks
drum sounds
bass drums
drum pads
drum beats
play drums
drum throne
electronic drums
play drum
playing drums
drum pad
drum mic
playing drum
drum sets
kick drums
drum patterns
drum mics
drum track
drum loops
hand drum
drum overheads
drum stick
drum hoop
sounding drum
drum beat
drum keys
drum hardware
real drums
professional drum
live drum
drum pedal
every drum
drum dial
recording drums
hand drums
drum parts
superior drum
digital drum
different drum
recording drum
drums sound
drum circles
rock drum
builtin drum
beginner drum
drum rack
ear drum
evans drum
frame drum
record drums
pearl drum
record drum
drum teacher
midi drum
alesis drum
pro drum
drums since
drum circle
sons drum
drums etc
drum sequencer
played drums
experienced drum
sampled drum
roland drum
drum thrones
drum triggers
bongo drum
drum bags


In [25]:
res_printer('sax', scores_stud)

sax
-----
alto sax
tenor sax
soprano sax
alto saxophone
tenor saxophone
sax player
bari sax
sax mouthpiece
saxophone mouthpiece
baritone sax
sax reeds
seydel sax
sax stand
sax players
saxophone stand
soprano saxophone
play sax
sax strap
pocket sax
playing sax
sax clarinet
sax case


Бонусная идея: если в самих названиях есть одинаковая часть - объединяем упоминания по ней, реплейсим все вариации одной и считаем так; если в названиях нет одинаковой части - смотрим с чем они встречаются в одном отзыве, если там есть более общее наименование продукта (например "Yamaha's set is the best one. These drums sound insane" - Yamaha's set = drums) заменяем им более частное название (общие наименования можно внести списком или попробовать вычислить по частоте вхождений) 